In [ ]:
# Importing the neccesary libraries

import pandas as pd
import numpy as np
import os
import math
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Function for getting the data from the files

def get_data(path):
  f = open(path, "r")
  theText = f.read()
  dataArray = []  
  for i in theText.split('\n'):   
    temp = []   
    for j in i.split(","):  
      if not j:
        continue
      temp.append(j)
    if not temp:
      continue
    dataArray.append(temp)
  return dataArray

In [ ]:
# Formating the training data to our liking

path = os.getcwd()
path = os.path.join(path, 'TrainingRatings.txt')
# training_data = get_data("/content/TrainingRatings.txt")
training_data = get_data(path)
temp = []
temp1 = []
for i in training_data:
  temp.append(int(i[0]))
  temp1.append(int(i[1]))

# Getting the unique user Ids in train data
users = list(set(temp1))
users.sort()
users = dict(zip(users, list(range(len(users)))))

# Getting the unique movie Ids in train data
movies = list(set(temp))
movies.sort()
movies = dict(zip(movies, list(range(len(movies)))))

train_data = np.zeros((len(users), len(movies)), dtype='int')
reviewed = [[-1] * 1 for i in range(len(users))]

for i in training_data:
  m = int(i[0])
  u = int(i[1])
  Rating = int(float(i[2]))
  movieIndex = movies[m]
  userIndex = users[u]
  train_data[userIndex][movieIndex] = Rating

  if Rating != 0:
    if reviewed[userIndex][0] != -1:
      reviewed[userIndex].append(movieIndex)
    else:
      reviewed[userIndex].append(movieIndex)
      reviewed[userIndex].remove(-1)

mean = train_data.mean(axis=1)
train_np = train_data - mean[:, np.newaxis]

In [ ]:
# Formatting the testing data to our liking

path = os.getcwd()
path = os.path.join(path, 'TestingRatings.txt')
# testing_data = get_data("/content/TestingRatings.txt")
testing_data = get_data(path)
temp = []
temp1 = []
for i in testing_data:
  temp1.append(int(i[1]))
  temp.append(int(i[0]))

# Getting the unique user Ids in test dat
userY = list(set(temp1))
userY.sort()
usersY = dict(zip(userY, list(range(len(userY)))))

# Storing the testing data that needs prediction
needPred = [[-1] * 1 for i in range(len(userY))]
count = 0
test_data = np.zeros((len(userY), len(movies)), dtype='int')

for i in testing_data:
  m = int(i[0])
  u = int(i[1])
  Rating = int(float(i[2]))
  movieIndex = movies[m]
  userIndex = usersY[u]

  if 0 != Rating:
    count += 1
    if (needPred[userIndex][0] != -1):
      needPred[userIndex].append(movieIndex)
    else:
      needPred[userIndex].append(movieIndex)
      needPred[userIndex].remove(-1)
    test_data[userIndex][movieIndex] = Rating

mean1 = test_data.mean(axis=1)
test_np = test_data - mean1[:, np.newaxis]

In [ ]:
# Calculating the weights using cosine_similarity function from sklearn

weights = cosine_similarity(train_np, test_np)

# Filtering the neighbours based on similarity for faster predictions
knn = []
for a in range(len(weights)):
  c = [] 
  for i in range(len(weights[a])):
    if weights[a][i] > 0.22:
        c.append(i)
  knn.append(c)

In [ ]:
# Predicting the ratings using the weights learned

pred = np.zeros((len(userY), len(movies)), dtype='int')
for u in userY:
  a = users[u]
  ind = usersY[u]
  for j in needPred[ind]:
    kappa = np.sum(weights[ind])
    s = 0
    if kappa != 0:
      kappa = np.reciprocal(kappa)
    for i in knn[a]:
      t = np.multiply(kappa, weights[i][j])
      c = train_data[i][j] - mean[i]
      s += np.multiply(t, c)
    prediction = mean[a] + s
    try:
      pred[ind][j] = prediction
    except Exception as e:
      pred[ind][j] = mean[a]

In [ ]:
# Evaluations

#Mean Absolute Error
diff = np.abs(pred - test_data)
sum1 = np.sum(diff)
err = np.sqrt(sum1 / count)
print('MAE: ', err)

#Root Mean Squared Error
diff = np.abs(np.square(pred) - np.square(test_data))
sum1 = np.sum(diff)
err = np.sqrt(sum1 / count)
print('RMSE: ', err)